<a href="https://colab.research.google.com/github/shiyu-cheng/6895_Visual_Sentiment_Prediction/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import torch.optim as optim
import time
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from collections import Counter
from sklearn.metrics import confusion_matrix
import pandas as pd
import itertools
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [ ]:
!unzip '/content/drive/MyDrive/data/zipdata.zip' -d '/content/drive/MyDrive/data/image_data'

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
drive.flush_and_unmount()

In [30]:
rmdir /content/drive/MyDrive/6895/data/.ipynb_checkpoints

rmdir: failed to remove '/content/drive/MyDrive/6895/data/.ipynb_checkpoints': No such file or directory


In [33]:
transform = transforms.Compose([
    transforms.Resize(48),        # Resize the image to 256x256 pixels
    #transforms.CenterCrop(224),   # Crop the center 224x224 pixels
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    # transforms.Normalize(         # Normalize the image
    #     mean=[0.485, 0.456, 0.406],
    #     std=[0.229, 0.224, 0.225]
    # )
])

# data_dir = '/content/drive/MyDrive/Colab Notebooks/image_data'
data_dir = '/content/drive/MyDrive/6895/data'
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [34]:
dataset

Dataset ImageFolder
    Number of datapoints: 971
    Root location: /content/drive/MyDrive/6895/data
    StandardTransform
Transform: Compose(
               Resize(size=48, interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [35]:
total_count=len(dataset)
train_count = int(0.7 * total_count) 
valid_count = total_count - train_count 
train_dataset, valid_dataset = torch.utils.data.random_split(dataset=dataset, lengths=[train_count, valid_count])
batch_size=16
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
valid_dataset_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False) 
dataloaders = {'train': train_dataset_loader, 'valid': valid_dataset_loader}

In [36]:
def train_model(model, dataloaders, criterion, optimizer,device, num_epochs=50):
    since = time.time()
    best_acc = 0
    model.to(device)

    val_acc_history = []
    train_acc_history = []

    test_acc_history = []
    train_losses = []
    valid_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

          
            for inputs, labels in dataloaders[phase]:
                #print(inputs.shape)
                inputs = inputs.to(device)
                labels = labels.to(device)

           
                optimizer.zero_grad()
            
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

               
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            
            time_elapsed = time.time() - since
   
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                valid_losses.append(epoch_loss)

            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)
            


    return model, val_acc_history, train_acc_history,valid_losses, train_losses


# define plot accuracy and loss
def plot_acc_loss(val_acc_history, train_acc_history,valid_losses, train_losses):
  val_acc=[]
  for i in val_acc_history:
    val_acc.append(i.cpu().data.numpy())
  train_acc=[]
  for i in train_acc_history:
    train_acc.append(i.cpu().data.numpy())
  test_acc=[]

  
  plt.subplot(1,2,1)
  plt.plot(range(len(val_acc)),val_acc,label='val_acc')
  plt.plot(range(len(train_acc)),train_acc,label='train_acc')
  plt.xlabel('epoch')
  plt.ylabel('acc')
  plt.legend()
  

  plt.subplot(1,2,2)
  plt.plot(range(len(valid_losses)),valid_losses,label='val_loss')
  plt.plot(range(len(train_losses)),train_losses,label='train_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.show()

In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [38]:
from torchvision import models
import torch.nn as nn
model = models.resnet50(pretrained=True)  #Use the resnet18 model
num_ftrs =model .fc.in_features #Modify the number of model categories
model.fc = nn.Sequential(nn.Dropout(0.6),nn.Linear(num_ftrs, 24))


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cost_fun = nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) 
model = model.to(device)
model, val_acc_history, train_acc_history,valid_losses, train_losses= train_model(model, dataloaders, cost_fun, optimizer, device,num_epochs=40)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/39
----------
train Loss: 0.9705 Acc: 0.5538
valid Loss: 0.7795 Acc: 0.6130
Epoch 1/39
----------
train Loss: 0.7076 Acc: 0.6495
valid Loss: 0.9561 Acc: 0.5479
Epoch 2/39
----------
train Loss: 0.7907 Acc: 0.6495
valid Loss: 0.6404 Acc: 0.6541
Epoch 3/39
----------
train Loss: 0.6819 Acc: 0.6966
valid Loss: 0.7843 Acc: 0.6267
Epoch 4/39
----------
train Loss: 0.6995 Acc: 0.6981
valid Loss: 0.7346 Acc: 0.6438
Epoch 5/39
----------
train Loss: 0.6016 Acc: 0.7216
valid Loss: 0.8173 Acc: 0.5788
Epoch 6/39
----------
train Loss: 0.6771 Acc: 0.6745
valid Loss: 0.7202 Acc: 0.6199
Epoch 7/39
----------
train Loss: 0.6800 Acc: 0.6701
valid Loss: 0.8747 Acc: 0.6096
Epoch 8/39
----------
train Loss: 0.6897 Acc: 0.6524
valid Loss: 1.1781 Acc: 0.5890
Epoch 9/39
----------
train Loss: 0.6938 Acc: 0.6716
valid Loss: 0.9770 Acc: 0.5993
Epoch 10/39
----------
train Loss: 0.6429 Acc: 0.7054
valid Loss: 0.7051 Acc: 0.6233
Epoch 11/39
----------
train Loss: 0.6670 Acc: 0.6878
valid Loss: 0.7519 Ac